In [4]:
# 연습문제 :서울시 응답소 사이트의 원순씨에게 바랍니다 게시판 크롤링하기
# https://eungdapso.seoul.go.kr/

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os   # 폴더 생성하기 위해 필요함

#Step 2. 사용자에게 크롤링에 필요한 정보를 입력 받습니다.
print("=" *100)
print("연습문제 : 서울시 응답소 게시판 크롤링하기")
print("=" *100)

query_txt = '서울시응답소'


cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)

f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://eungdapso.seoul.go.kr/')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

#Step 4. 여러 메뉴 중에서 원순씨에게 바랍니다 게시판 선택
driver.find_element_by_xpath("""//*[@id="gnb"]/div[4]/div[1]/div/ul/li[6]/a""").click()

#시간을 계산하기 위해 현재 타임스탬프 찍기
now1 = time.localtime()
s1 = '%04d-%02d-%02d-%02d:%02d:%02d' % (now1.tm_year, now1.tm_mon, now1.tm_mday, now1.tm_hour, now1.tm_min, now1.tm_sec)

print("\n")
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)

# 페이지 번호 만들기
page_no=[]

for i in range(5,cnt) :
    if i % 5 == 0 :
            page_no.append(i + 1)
                
# Step 5.게시물의 정보 수집

no2=[]             # 게시글 번호 컬럼
contents=[]        # 게시글 내용 컬럼

click_cnt=0

no = 1

for x in range(1,page_cnt+1) :
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
        
    # 1 페이지에 있는 게시글 10건의 상세 내역 가져오기
    for i in range(1,11) :
        
        if no > cnt :
            break
            
        print
        f = open(ff_name, 'a',encoding='UTF-8')
        
        driver.find_element_by_xpath("""//*[@id="content_cont"]/div[2]/div/form/div[4]/div[%s]/ul/li[2]/a"""  %i).click( )
                               
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        content = soup.find('div','table_inner_desc').get_text().replace("\t","")
        f.write(str(no)+':'+str(content)+'\n')
        f.close( )
        
        print(no,':',content)
   
        no2.append(no)                             # 게시글 번호에 추가
        contents.append(content.replace("\n","")+'\n')   # 내용에 추가
        
        no += 1
        
        time.sleep(1)
        
        driver.back()
        
    time.sleep(2)        # 페이지 변경 전 2초 대기 

    i = math.floor(x/5) -1
    
    if x > page_cnt+1 :
        break           
    
    if no > cnt :
        break
    elif x == page_no[i]:
        driver.find_element_by_link_text('''다음''').click()
    else :
        driver.find_element_by_link_text("""%s""" %x).click() # 다음 페이지번호 클릭

    
# Step 6. 수집된 정보를 다양한 형식의 파일로 저장하기 
seoul = pd.DataFrame()
seoul['번호']=no2
seoul['상담내용']=contents

# csv 형태로 저장하기
seoul.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
seoul.to_excel(fx_name,index=False)

#Step 7. 요약 정보 출력하기
e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *100)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

driver.close( )

연습문제 : 서울시 응답소 게시판 크롤링하기
1.크롤링 할 건수는 몇건입니까?: 60
2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\temp\):c:\temp\


크롤링 할 총 페이지 번호:  6
1 페이지 내용 수집 시작합니다 =======================
1 : 
Q. 상담내용
먼저 강북균형발전위해 애써주신 시장님께 감사드립니다Sh본부 중랑구 이전 결정에 대해 노조에서는 sh본사가 신내동으로 이전 시 교통이 불편해서 sh를 찾는 고객 및 직원들의 불편이 예상되어 반대한다고 하였습니다현재 sh본사 이전예정위치는 신내역 6호선 경춘선이 도보로 7분, 추후 생길 면목선역은 그보다 훨씬 가깝습니다버스로 조금만나가도 망우역과 상봉역이 있습니다교통때문에 오기 힘들다는 노조의 주장은 타당성이나 근거가 부족합니다그리고 저도 전 직장을 공공기관에 있던 사람으로써 해당 직장본사도 공덕에 있다가 그나마 지방으로 안가고 송내로 가게되어 다행이었다고 하는 상황인데 이 기관은 강북 신내로 오게 된것이면 나쁘지 않은 조건이라 생각합니다？무엇보다 신내는 현재 300인 이상기업이 하나라도 있는지 의심스러울정도로 열악한 베드타운이나 근래에 양원지구 및 데시앙플렉스등 기업유치를 위한 노력을 해나가는 단계입니다 이럴때 sh공사 본사가 온다는 소식은 이런노력들의 매우 큰 보탬이 될것이며 시너지를 가져 올것입니다분명 공사본사와 신내동에 윈윈이 될것이라 의심치 않습니다그럴리 없겠지만 노조의 압력에 굴하지 말아주시고 꼭 확정하신것처럼 시행해주시기를 요청드립니다

2 : 
Q. 상담내용
1. 5호선연장과 차량기지 이전은 당연한 것이고...건폐장 이전도 반드시 동시에 진행되어야 합니다. 건폐장 이전 대책이 없다면 서울시의 무능함으로 밖에는 보이지 않습니다.2. 서울에서 강서구 지역만...특히 방화동은 현재 박원순시장님께서 고의적으로 환경 및 시설을 낙후시키고자 의도하는게 아닌가 하는 의심이 들 정도로 아무런 대책도 개선도 이루어지지 않고 있습니다.3. 2번 연속해서 박시장님께 표를 던진

2 페이지 내용 수집 시작합니다 =======================
11 : 
Q. 상담내용
먼저 강북균형발전위해 애써주신 시장님께 감사드립니다Sh본부 중랑구 이전 결정에 대해 노조에서는 sh본사가 신내동으로 이전 시 교통이 불편해서 sh를 찾는 고객 및 직원들의 불편이 예상되어 반대한다고 하였습니다현재 sh본사 이전예정위치는 신내역 6호선 경춘선이 도보로 7분, 추후 생길 면목선역은 그보다 훨씬 가깝습니다버스로 조금만나가도 망우역과 상봉역이 있습니다교통때문에 오기 힘들다는 노조의 주장은 타당성이나 근거가 부족합니다그리고 저도 전 직장을 공공기관에 있던 사람으로써 해당 직장본사도 공덕에 있다가 그나마 지방으로 안가고 송내로 가게되어 다행이었다고 하는 상황인데 이 기관은 강북 신내로 오게 된것이면 나쁘지 않은 조건이라 생각합니다？무엇보다 신내는 현재 300인 이상기업이 하나라도 있는지 의심스러울정도로 열악한 베드타운이나 근래에 양원지구 및 데시앙플렉스등 기업유치를 위한 노력을 해나가는 단계입니다 이럴때 sh공사 본사가 온다는 소식은 이런노력들의 매우 큰 보탬이 될것이며 시너지를 가져 올것입니다분명 공사본사와 신내동에 윈윈이 될것이라 의심치 않습니다그럴리 없겠지만 노조의 압력에 굴하지 말아주시고 꼭 확정하신것처럼 시행해주시기를 요청드립니다

12 : 
Q. 상담내용
1. 5호선연장과 차량기지 이전은 당연한 것이고...건폐장 이전도 반드시 동시에 진행되어야 합니다. 건폐장 이전 대책이 없다면 서울시의 무능함으로 밖에는 보이지 않습니다.2. 서울에서 강서구 지역만...특히 방화동은 현재 박원순시장님께서 고의적으로 환경 및 시설을 낙후시키고자 의도하는게 아닌가 하는 의심이 들 정도로 아무런 대책도 개선도 이루어지지 않고 있습니다.3. 2번 연속해서 박시장님께 표를 던진 서울시민에게 부디 배신감이 들지 않도록 해주시기를 간곡히 당부드립니다!!!

13 : 
Q. 상담내용
DPF 매연저감장치 신청후 대기중인 차량의 자동차는매연저감장치 설치후 자동차 검사받을수 있도록

23 : 
Q. 상담내용
저희 아이가 하루 종일 구석에 갇혀 있도록한 교사들과 원장을 신고합니다..다시는 저희 아이와 같은 분통하고 마음 아픈 일이 생기지 않도록..피해받는 조그맣고 소중한 아이들이 없길.. 간절하고 간절히 바랍니다..

24 : 
Q. 상담내용
수도 서울  아리수는 누가 먹어도 깨끗하고 오염되지 않아야합니다 그런데 지구상에 녹찌꺼기 가 나오는 물이 아리수라하면 아니다 라고 믿지않을 할텐데 우리집 아리수는 녹찌꺼기가 더덕더덕붙여서나오는 아리수를 먹고있는데 중부수도 사업소에서는 법조항 만 앞세우고 수용가 책임이라는 책임전가 에 만 급급하는데 이제는 말도 바꿔 가면서 전에는 총 공사비의 80%를 보조해 준다 했는데 이제는 오히려 일부라고 말바꾸기나 하고 민원해결 방안을 찾는게 아니고 소극적인 태도로 민원제기한 횟수나세고 있는 한심한 조직이 수도 서울 식수를 제공한다는 중부수도사업소 현장민원팀 인데 현장에 직접나와서 민원인과 대면 한번 없이 일방적인 민원 처리완료 나 하고 무사안일한 자세로 근무를 하는데 관계당국에서 는 이런 무사안일한 현장 민원과 의 사실을 전혀 보르고 있는데아니 민원 접수를 하면 중간에서 차단해 버리서 관계당국 시장님 직속 부서에 가기도 전에 민원 차단을 한것 같은데  적극적인 민원처가 이루워 질 수있도록 처리하여주시기 바랍니다.

25 : 
Q. 상담내용
2002년 11월 서울 양천구 목동중앙본로7길 37-20 목동길성그랑프리텔아파트에 입주해 거주하고 있는 시민입니다. 다름아니라, 저희가 처음 입주했을 당시에는 목2동-목4동 방향에서 대일고 쪽의 등촌로로 다니는 마을버스가 있었습니다. 그런데 시간이 지나면서 목2동 강서보건소에서 당산,오목교 방향의 노선만 운행되고 반대방향의 노선은 없어진 듯 합니다. 대일고,영일고에 진학해서 다니는 학생들도 상당하고, 시민들도 상당히 있었는데 왜 노선이 폐지되었는지 모르겠습니다. 목2동에서 목3,4동을 거쳐 대일고 등촌로로 이어지는 마을버스 노선의 회복을 검토 요청하고자 합니다. 주변 상권도 활

40 : 
Q. 상담내용
부득이 한강 여의도 선상시위를 하게 되었습니다.

5 페이지 내용 수집 시작합니다 =======================
41 : 
Q. 상담내용
안녕하세요 날씨도 더운데 수고하시네요문의를 할께 있어서요 내손안에 서울  포토사진을 올리고 싶은데 어떻게하면되나요 이전에는 사진 올리기가  편했는데 지금은 사진올리기가 힘드네요

42 : 
Q. 상담내용
올해 5월말 지인이 아파트베란다에 300W태양광을 설치했습니다. 서울에너지공사 연계홈페이지"햇빛마루"에 의하면 300W월평균 태양광 발전량 약29.2KWh.월절감액6,190원,연간절감액 7,4280원 그럭저럭 의미있는 수치다. 그런데 이수치는 남향 최고의조건에서 생산되는 수치라는것이 문제다.동.서향에서는 그 수치가 현저히 떨어진다. 지인의경우 한달평균 10KWh도 생산이 안되었다.그것도 1년중 가장 효율이 좋다는4,5,6월중 한달에도 말이다.그래서 응답소를 통해" 동.서향에 대한 축적된 데이터가 있느냐?"문의 했지만 "축적된 데이터는 없고 올 하반기부터 사물인터넷을 통한 데이터 축적을 위한 준비를 한다."고 한다. 답답했다. 데이터 축적은 설치한 가구에 1년에 한,두번 전화나 문자를 통하여 인버터의 누적생산량을 확인하면 간단하게 수집 할 수 있는데 시행6년간 준비기간까지 포함하면 더 많은 시간이 흘렀을 텐데,생산량이 현저하게 떨어지는 동.서향에 대한 데이터가 없다는것이 의문이다. 연도별 달성계획만 있고 생산성이나 효율은 생각하지 않는 전형적인 탁상 행정이다.오히려 남향 최대생산량보다 동.서향에대한 데이터 축적이 우선되어야 하지 않았을까?어디에도 답이없던 동.서향 데이터를 조금이나마 근접하게 찾을 수 있었다.응답소에 몇번의 문의와 통화를 하면서 받은 자료를 통해 조사해 봤다.물론 수치화된건 없었으나 "서울시햇빛지도"에서 "햇빛주제도조회"에서 지인의 아파트를 조회해보니 분기별로 4등급과5등급으로 표기된다.1등급을 100으로 볼때5등급은 반이 조금 넘는 수치다. 그나마 앞동과의 거리나 층수에따

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content_cont"]/div[2]/div/form/div[4]/div[1]/ul/li[2]/a"}
  (Session info: chrome=78.0.3904.108)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18362 x86_64)
